# Notebook for IBM capstone Project 

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import json 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 


# Data Preprocessing 

In [2]:
#Importing the table from the wiki page
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.columns = ['Postal Code','Borough','Neighborhood']
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


In [3]:
#Dropping rows that are not associated to any Borough 
NA = df[df['Borough']=='Not assigned'].index
df.drop(NA,inplace=True)
df

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


In [4]:
#Checking to see if any Borough are given to not assigned neighbourhoods 
neigh = df[df['Neighborhood']=='Not assigned']
neigh

,Postal Code,Borough,Neighborhood


In [5]:
#4Square API Login (Used later to pulll venue locations for Kmeans)
CLIENT_ID = 'TSVZ0QHSX0QDIVJTP4L3O4CNFSSU2WP0PENXI1F04502OIHK' # your Foursquare ID
CLIENT_SECRET = 'BPRI1ROATI03W15Y5KQCQMYZ0C2TS4LWWJCL0IWGNG32IIY2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [6]:
#Reading in the Lat/Long coords for M postal Codes in canada 
df_coords = pd.read_csv('http://cocl.us/Geospatial_data')
df_coords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [7]:
#Merging the 2 DF's based on Postal Code 
df_pos = pd.merge(df, df_coords, on=['Postal Code'], how='inner')
df_tor = df_pos[['Borough', 'Neighborhood', 'Postal Code', 'Latitude', 'Longitude']].copy()
df_tor

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,Regent Park / Harbourfront,M5A,43.654260,-79.360636
3,North York,Lawrence Manor / Lawrence Heights,M6A,43.718518,-79.464763
4,Downtown Toronto,Queen's Park / Ontario Provincial Government,M7A,43.662301,-79.389494
...,...,...,...,...,...
98,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,M8X,43.653654,-79.506944
99,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
100,East Toronto,Business reply mail Processing CentrE,M7Y,43.662744,-79.321558
101,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,M8Y,43.636258,-79.498509


In [8]:
df_tor.shape

(103, 5)

# Clustering the neighbourhoods

Using folium to map out the location of each neighbourhood on the map and then using k means to create clusters based off of venue type using the top 10 veneus from each neighbourhood

In [9]:
#finding the lat/long coords of toronto
address = 'Toronto, Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


In [10]:
#Creating a map of Toronto With the Neighbourhoods ontop 
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
LIMIT=100

The following function is used in order to pull venue information from 4square and store it in order to be able to perform segmentation 

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
#Pulling venues and assiging them to their neighbourhood 
tor_ven = getNearbyVenues(names=df_tor['Neighborhood'],
                                   latitudes=df_tor['Latitude'],
                                   longitudes=df_tor['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

In [16]:
#sanity checking the dataframe created 
tor_ven.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
1,Regent Park / Harbourfront,43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
2,Regent Park / Harbourfront,43.654260,-79.360636,Terroni Sud Forno Produzione e Spaccio,43.653903,-79.360018,Gourmet Shop
3,Regent Park / Harbourfront,43.654260,-79.360636,Massimo's Kitchen Studio,43.654770,-79.359698,Italian Restaurant
4,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,Roots,43.718214,-79.463893,Boutique


In [17]:
print('There are {} uniques categories.'.format(len(tor_ven['Venue Category'].unique())))

There are 75 uniques categories.


In [18]:
#Using 1 hot encoding to set up dataframe
toronto_onehot = pd.get_dummies(tor_ven[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = tor_ven['Neighborhood'] 

# move neighborhood column to the first column
neigh = toronto_onehot['Neighborhood']
toronto_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
toronto_onehot.insert(0, 'Neighborhood', neigh)

toronto_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Beer Store,Bookstore,...,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Lawrence Manor / Lawrence Heights,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Beer Store,Bookstore,...,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,Bathurst Manor / Wilson Heights / Downsview North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Bedford Park / Lawrence Manor East,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,CN Tower / King and Spadina / Railway Lands / ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Canada Post Gateway Processing Centre,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


The following function is isued to find the most common veneue from each neighbourhood

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
#Creating a dataframe with the top 10 most common venues for each neighbourhood
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Deli / Bodega,Fried Chicken Joint
1,Bedford Park / Lawrence Manor East,Coffee Shop,Italian Restaurant,Sushi Restaurant
2,CN Tower / King and Spadina / Railway Lands / ...,Performing Arts Venue,Yoga Studio,Fast Food Restaurant
3,Canada Post Gateway Processing Centre,Coffee Shop,Cocktail Bar,College Gym
4,Central Bay Street,Coffee Shop,Pharmacy,Sandwich Place


In [22]:
#Performing Kmeans clustering 
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 1, 1, 3, 3, 1, 1, 1, 1, 1])

In [23]:
#Merging the clustering labels into the main DF
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_tor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()


,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,North York,Parkwoods,M3A,43.753259,-79.329656,NaN,NaN,NaN,NaN
1,North York,Victoria Village,M4A,43.725882,-79.315572,NaN,NaN,NaN,NaN
2,Downtown Toronto,Regent Park / Harbourfront,M5A,43.654260,-79.360636,1.0,Gourmet Shop,Italian Restaurant,Breakfast Spot
3,North York,Lawrence Manor / Lawrence Heights,M6A,43.718518,-79.464763,1.0,Clothing Store,Boutique,Hot Dog Joint
4,Downtown Toronto,Queen's Park / Ontario Provincial Government,M7A,43.662301,-79.389494,NaN,NaN,NaN,NaN


In [24]:
#Certain neighbourhoods were returning a NaN value for their cluster label so they are assigned an abitrary llabel of 5
toronto_merged['Cluster Labels'].fillna(5,inplace=True)

In [25]:
#Transforming the cvluster labels from float to Int 
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype(int)

In [26]:
# create map
latitude, longitude = 43.6532, -79.3832
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters